In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random
import shutil
%matplotlib inline

!ls
%mkdir data
!ls

sample_data
data  sample_data


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cargar los datos
shutil.copy("/content/drive/MyDrive/sis_420/lab_repaso/01_regresion/GoldUP.csv","/content/data/data.csv")

datos = pd.read_csv( os.path.join('data', 'data.csv') )
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           239 non-null    object 
 1   Gold_Price     239 non-null    int64  
 2   Crude_Oil      239 non-null    float64
 3   Interest_Rate  239 non-null    float64
 4   USD_INR        239 non-null    float64
 5   Sensex         239 non-null    float64
 6   CPI            239 non-null    float64
 7   USD_Index      239 non-null    float64
dtypes: float64(6), int64(1), object(1)
memory usage: 15.1+ KB


In [4]:
le = LabelEncoder()
datos['Date'] = le.fit_transform(datos['Date'])
col = datos.pop("Gold_Price")
datos.insert(len(datos.columns), "Gold_Price", col)


In [5]:
X = datos.iloc[:, :7].values #Las primeras siete columnas, que son nuestras X
y = datos.iloc[:, 7].values.reshape(-1, 1) #La ultima columna, Que es nuestra y

In [6]:
#creamos instacias de la clase para normalizar las x, y
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Normalizar las características (X). fit_transform ajusta el escalador y transforma los datos.
X = scaler_x.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1))

In [7]:
# Dividimos los datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Convierte los datos en tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.float32)

In [8]:
# Define el modelo de regresión lineal multivariable
class LinearRegression(nn.Module):
    def __init__(self, input_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, 1)  # Salida: una dimensión

    #este metodo describe como los datos de entrada se propagan en este caso, e la capa lineal
    def forward(self, x):
        return self.linear(x)

In [9]:
# obtenemos la dimension de la matriz que seria el numero de caracteristicas del modelo
input_size = X_train.shape[1]
model = LinearRegression(input_size) # instanciamos la clase y le pasamos la longutudad

# Función de pérdida y optimizador
criterion = nn.MSELoss() #instanciamos la funcion de perdida, para calcular la pérdida asociada con las predicciones

# usammos el metodo SGD para ajustar los parámetros de un modelo durante el entrenamiento
# crea un optimizador utilizando el descenso de gradiente
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [10]:
# Cantidad de ciclos de entrenamiento
num_epochs = 2000
for epoch in range(num_epochs):
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Evalúa el modelo en el conjunto de prueba
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Pérdida en el conjunto de prueba: {test_loss.item():.4f}')

Epoch [200/2000], Loss: 0.0912
Epoch [400/2000], Loss: 0.0804
Epoch [600/2000], Loss: 0.0737
Epoch [800/2000], Loss: 0.0691
Epoch [1000/2000], Loss: 0.0657
Epoch [1200/2000], Loss: 0.0631
Epoch [1400/2000], Loss: 0.0609
Epoch [1600/2000], Loss: 0.0590
Epoch [1800/2000], Loss: 0.0574
Epoch [2000/2000], Loss: 0.0559
Pérdida en el conjunto de prueba: 0.0538


In [11]:
# Pruebas
# Número de pruebas que deseas mostrar
cant_pruebas = 15

# Obtén índices aleatorios de ejemplos en el conjunto de prueba
random_indices = np.random.randint(0, len(X_test), size=cant_pruebas)

print('Nº Test | Precio Real | Predecido')
for i in random_indices:
    # Obtiene el precio real y predecido, revirtiendo la transformación de escala
    current_observation = y_test[i].reshape(-1, 1)
    real_value = scaler_y.inverse_transform(current_observation)[0][0]

    # Obtén la predicción actual del modelo
    current_prediction = y_pred[i].reshape(-1, 1)
    predicted_value = scaler_y.inverse_transform(current_prediction)[0][0]

    # Imprime la información formateada
    print(f'{i + 1 }  | {real_value: .2f}     | {predicted_value: .2f}')

Nº Test | Precio Real | Predecido
38  |  26246.00     |  28242.42
39  |  5771.00     |  6899.88
27  |  30538.00     |  37133.33
32  |  37927.00     |  36939.22
6  |  6874.00     |  9066.36
29  |  4267.00     |  2578.91
15  |  5718.00     |  6472.69
40  |  6059.00     |  8090.05
47  |  6149.00     |  8628.31
16  |  4541.00     |  2235.64
17  |  38092.00     |  38036.17
32  |  37927.00     |  36939.22
10  |  26115.00     |  27945.04
43  |  18270.00     |  17095.61
10  |  26115.00     |  27945.04
